In [1]:
import openpyxl
import numpy
import scipy
import numpy as np
import pandas as pd
import FeatureExtract # 自己提取时域频域特征的方法实现

In [2]:
raw_data = openpyxl.load_workbook(u'/home/chris/predict_code/lyft/bci/data/S1/S1_train_data.xlsx')
raw_event = openpyxl.load_workbook(u'/home/chris/predict_code/lyft/bci/data/S1/S1_train_event.xlsx')
print(raw_data.sheetnames)
print(raw_event.sheetnames)

['char01(B)', 'char02(D)', 'char03(G)', 'char04(L)', 'char05(O)', 'char06(Q)', 'char07(S)', 'char08(V)', 'char09(Z)', 'char10(4)', 'char11(7)', 'char12(9)']
['char01(B)', 'char02(D)', 'char03(G)', 'char04(L)', 'char05(O)', 'char06(Q)', 'char07(S)', 'char08(V)', 'char09(Z)', 'char10(4)', 'char11(7)', 'char12(9)']


In [3]:
# 行列标识符
chars = {'A': (1, 7), 'B': (1, 8), 'C': (1, 9), 'D': (1, 10), 'E': (1, 11), 'F': (1, 12), 
         'G': (2, 7), 'H': (2, 8), 'I': (2, 9), 'J': (2, 10), 'K': (2, 11), 'L': (2, 12), 
         'M': (3, 7), 'N': (3, 8), 'O': (3, 9), 'P': (3, 10), 'Q': (3, 11), 'R': (3, 12), 
         'S': (4, 7), 'T': (4, 8), 'U': (4, 9), 'V': (4, 10), 'W': (4, 11), 'X': (4, 12), 
         'Y': (5, 7), 'Z': (5, 8), '1': (5, 9), '2': (5, 10), '3': (5, 11), '4': (5, 12), 
         '5': (6, 7), '6': (6, 8), '7': (6, 9), '8': (6, 10), '9': (6, 11), '0': (6, 12), 
        }

In [9]:
# 脑电数据集转换为P300二分类数据集，含数据预处理、滤波、特征提取等
classify_dataset = []

# 带通滤波器上下界
freq_min, freq_max = 1, 30
# 调用8阶带通滤波器
# b, a = scipy.signal.butter(8, [0.02,0.8], 'bandpass')

for sheet in raw_event.sheetnames:
    # 当前字符
    current_char = sheet[7]
    # print(current_char)
    table_event = raw_event[sheet]
    table_data = raw_data[sheet]
    for row in range(raw_event[sheet].max_row):
        if table_event.cell(row+1, 1).value >= 1 and table_event.cell(row+1, 1).value <= 12:
            # data.shape: 275,-100ms~1000ms时间窗口20个电压的均值
            # feature: 26
            feature = []
            data = []
            all_data = []
            for i in range(20):
                channel = []
                for j in range(275):
                    channel.append(table_data.cell(table_event.cell(row+1, 2).value-24+j, i+1).value)
                all_data.append(channel)
                for pole in range(len(all_data)):
                    data.append(np.max(all_data[pole],axis=0))
            
            # 带通滤波
            # filter_data = scipy.signal.filtfilt(b, a, data[][i])
            filter_data = data
            # 16个时域特征
            feature.append(FeatureExtract.mean_fea(filter_data))
            feature.append(FeatureExtract.rms_fea(filter_data))
            feature.append(FeatureExtract.sr_fea(filter_data))
            feature.append(FeatureExtract.am_fea(filter_data))
            feature.append(FeatureExtract.skew_fea(filter_data))
            feature.append(FeatureExtract.kurt_fea(filter_data))
            feature.append(FeatureExtract.max_fea(filter_data))
            feature.append(FeatureExtract.min_fea(filter_data))
            feature.append(FeatureExtract.pp_fea(filter_data))
            feature.append(FeatureExtract.var_fea(filter_data))
            feature.append(FeatureExtract.waveform_index(filter_data))
            feature.append(FeatureExtract.peak_index(filter_data))
            feature.append(FeatureExtract.impluse_factor(filter_data))
            feature.append(FeatureExtract.tolerance_index(filter_data))
            feature.append(FeatureExtract.skew_index(filter_data))
            feature.append(FeatureExtract.kurt_index(filter_data))
            # 10个频域特征
            feature.append(FeatureExtract.fft_mean(filter_data))
            feature.append(FeatureExtract.fft_var(filter_data))
            feature.append(FeatureExtract.fft_std(filter_data))
            feature.append(FeatureExtract.fft_entropy(filter_data))
            feature.append(FeatureExtract.fft_energy(filter_data))
            feature.append(FeatureExtract.fft_skew(filter_data))
            feature.append(FeatureExtract.fft_shape_mean(filter_data))
            feature.append(FeatureExtract.fft_shape_std(filter_data))
            feature.append(FeatureExtract.fft_shape_skew(filter_data))
            feature.append(FeatureExtract.fft_shape_kurt(filter_data))
            if table_event.cell(row+1, 1).value == chars[current_char][0] or table_event.cell(row+1, 1).value == chars[current_char][1]:
                feature.append(table_event.cell(row+1, 1).value)
            else:
                feature.append(0)
            classify_dataset.append(feature)
print((np.array(classify_dataset)).shape)
df = pd.DataFrame(classify_dataset)
df.to_csv('s1_avg_train.csv')

(720, 27)
